In [ ]:
'''
对某个站点的 运行 进行健康评估
输入：站点名称site_id
输出：该站点所有设备的告警分数 score_dict 以及该站点的告警总分数 score_total
'''

import numpy as np 
import datetime

# 输入：站点名称site_id
# 输出：该站点的告警总分数score_total

def alert_score(site_id,equip_data,tank_alerts,now_date,check_arrange_day,alert_1,alert_2,alert_3):
    score_dict = {}
    score_arr = []
    ### 设备集合中，有可能设备没有id号，是NAN值，这时默认是一个设备，要加入分数计算，但无需计算权重
    device_data = equip_data[equip_data['site_id'] == site_id]['device_id'].unique()
    print(device_data)
    ### 对每个设备进行告警分数计算
    for i in range(len(device_data)):
        alerts_data = tank_alerts[tank_alerts['device_id'] == device_data[i]]
        end_date = str(now_date)
        start_date = str(now_date - datetime.timedelta(days = check_arrange_day))
        alerts_range_data = alerts_data[(alerts_data['start_time'] < end_date) 
                                    & (start_date < alerts_data['start_time'])]
        level_id = alerts_range_data['level'].values
        n1,n2,n3 = 0,0,0
        for j in level_id:
            if j == 'alarm':
                n1 += 1
            elif j == 'critical':
                n2 += 1
            elif j == 'major':
                n3 += 1
        score_dict[str(device_data[i])] = (max(0,100 - alert_1*n1 - alert_2*n2 - alert_3*n3))
        score_arr.append(max(0,100 - alert_1*n1 - alert_2*n2 - alert_3*n3))
    score_total = np.array(score_arr).sum()/(len(score_arr))
    return score_total